In [10]:
import pandas as pd
import copy
import os
import numpy as np
import copy

# Functions

In [11]:
distances = [
    100,
    200,
    300,
    400,
    500,
    600,
    -1
]

In [12]:
def concat_results(result_name):
    file_names = []
    for filename in os.listdir():
        if result_name in filename and result_name+".csv"!=filename:
            file_names.append(filename)

    csv_file = pd.concat([pd.read_csv(file) for file in file_names])
    
    if "wheather" in csv_file.columns.tolist():
        csv_file.rename(columns = {"wheather": "weather"}, inplace = True)
    
    csv_file.to_csv(result_name + ".csv", index=False)
    
    print("find files")
    print(file_names)
    print("write new csv")
    print(result_name + ".csv")
    print()

def string_to_dict(string_):
    string = copy.deepcopy(string_)
    string = string.replace("array", "np.array")
    string = string.replace("inf", "10000")
    string = string.replace("nan", "-1")
    return eval(string)

def add_mean(result):
    global distances
    for i in result.keys():
        vals = [result[i][j] for j in distances]
        result[i]["mean"] = sum(vals)/len(vals)
    return result

def print_result(result, multiplier=100, round_num=None):
    global distances
    row_format = "{:>20}" * (len(result.keys()) + 1)
    print(row_format.format("", *result.keys()))
    
    for dist in distances+["mean"]:
        row = [round(result[conditions][dist]*multiplier, round_num) for conditions in result.keys()]
        print(row_format.format(dist, *row))
    
    print("-"*100)
    for dist in ["out_of_track", "is_collision", "nospeed"]:
        row = [round(result[conditions][dist]*multiplier, round_num) for conditions in result.keys()]
        print(row_format.format(dist, *row))

In [13]:
def get_result_by_conditions(result_csv, result_key = "success"):
    
    
    training_conditions = {
        "city_name": ["Offroad_1"],
        "weather": ["ClearNoon"],
    }

    in_keys = {
        100: [],
        200: [],
        300: [],
        400: [],
        500: [],
        600: [],
        -1: [],
        "out_of_track": [],
        "is_collision": [],
        "nospeed": []
    }

    result = {
        "training_conditions": copy.deepcopy(in_keys),
        "new_weather": copy.deepcopy(in_keys),
        "new_city": copy.deepcopy(in_keys),
        "new_city_weather": copy.deepcopy(in_keys)
    }

    for i in range(len(result_csv)):
        line = result_csv.iloc[i]
        

        same_city = line["city_name"] in training_conditions["city_name"]
        same_weather = line["weather"] in training_conditions["weather"]
        dist = int(line["lane_distance"])
        
        condition = ""
        if same_city and same_weather:
            condition = "training_conditions"
        elif not same_city and same_weather:
            condition = "new_city"
        elif same_city and not same_weather:
            condition = "new_weather"
        elif not same_city and not same_weather:
            condition = "new_city_weather"
        
        epinfos = string_to_dict(line["epinfos"])
        
        success = int(line["success"])
        epinfos["success"] = success
        result[condition][dist].append(epinfos[result_key])
        
        out_of_track = epinfos["track_width"]/2 < epinfos["distance_from_center"] and epinfos["distance_from_center"]!=-1
        is_collision = epinfos["is_collision"]
        nospeed = epinfos["nospeedtime_step"] > 20
        
        if not success:
            result[condition]["out_of_track"].append(out_of_track)
            result[condition]["is_collision"].append(is_collision)
            result[condition]["nospeed"].append(nospeed)
        
    for i in result.keys():
        for j in result[i].keys():
            try:
                result[i][j] = sum(result[i][j])/len(result[i][j])
            except ZeroDivisionError:
                result[i][j] = 0
                print("no data", i, j)
    return result

def get_result_by_offroads(result_csv, result_key="success"):
    training_conditions = {
        "city_name": ["Offroad_1"],
        "weather": ["ClearNoon"],
    }

    in_keys = {
        100: [],
        200: [],
        300: [],
        400: [],
        500: [],
        600: [],
        -1: [],
        "out_of_track": [],
        "is_collision": [],
        "nospeed": []
    }

    result = {
        "Offroad_1": copy.deepcopy(in_keys),
        "Offroad_2": copy.deepcopy(in_keys),
        "Offroad_3": copy.deepcopy(in_keys),
        "Offroad_4": copy.deepcopy(in_keys),
        "Offroad_5": copy.deepcopy(in_keys),
        "Offroad_6": copy.deepcopy(in_keys),
        "Offroad_7": copy.deepcopy(in_keys),
        "Offroad_8": copy.deepcopy(in_keys)
    }

    for i in range(len(result_csv)):
        line = result_csv.iloc[i]

        dist = int(line["lane_distance"])
        
        success = int(line["success"])
        epinfos = string_to_dict(line["epinfos"])
        epinfos["success"] = success
        
        result[line["city_name"]][dist].append(epinfos[result_key])
        
        out_of_track = epinfos["track_width"]/2 < epinfos["distance_from_center"] and epinfos["distance_from_center"]!=-1
        is_collision = epinfos["is_collision"]
        nospeed = epinfos["nospeedtime_step"] > 20
        
        if not success:
            result[line["city_name"]]["out_of_track"].append(out_of_track)
            result[line["city_name"]]["is_collision"].append(is_collision)
            result[line["city_name"]]["nospeed"].append(nospeed)
        
    for i in result.keys():
        for j in result[i].keys():
            try:
                result[i][j] = sum(result[i][j])/len(result[i][j])
            except ZeroDivisionError:
                result[i][j] = 0
                print("no data", i, j)
    return result

# Success percentage by Conditions

In [14]:
results = [
    "sample_result"
]

In [15]:
for name in results:
    csv_file = pd.read_csv(name+".csv")
    result = get_result_by_conditions(csv_file, result_key="success")
    result = add_mean(result)
    
    print(name)
    print()
    print_result(result, multiplier=100, round_num=None)
    
    try:
        os.mkdir("Success percentage by Conditions")
    except:
        pass
    result = pd.DataFrame(result)
    result *= 100
    result = result.astype(int)
    result.to_csv(f"Success percentage by Conditions/{name}.csv")
    print()

no data training_conditions out_of_track
no data training_conditions is_collision
no data training_conditions nospeed
sample_result

                     training_conditions         new_weather            new_city    new_city_weather
                 100                 100                 100                  33                  32
                 200                 100                 100                  24                  23
                 300                 100                 100                  17                  24
                 400                 100                 100                  23                  20
                 500                 100                  98                  14                  20
                 600                 100                 100                  17                  19
                  -1                 100                 100                   0                   0
                mean                 100                 10

# Success percentage by Offroads

In [16]:
for name in results:
    csv_file = pd.read_csv(name+".csv")
    result = get_result_by_offroads(csv_file, result_key="success")
    result = add_mean(result)
    
    print(name)
    print()
    print_result(result, multiplier=100, round_num=None)
    
    try:
        os.mkdir("Success percentage by Offroads")
    except:
        pass
    result = pd.DataFrame(result)
    result *= 100
    result = result.astype(int)
    result.to_csv(f"Success percentage by Offroads/{name}.csv")
    print()

sample_result

                               Offroad_1           Offroad_2           Offroad_3           Offroad_4           Offroad_5           Offroad_6           Offroad_7           Offroad_8
                 100                 100                  45                  87                  93                   0                   0                   0                   0
                 200                 100                   3                  75                  85                   0                   0                   0                   0
                 300                 100                   3                  77                  82                   0                   0                   0                   0
                 400                 100                   0                  77                  68                   0                   0                   0                   0
                 500                  98                   0                  67

# Average speed by Conditions

In [17]:
keys = ["average_speed" for _ in range(len(results))]

In [18]:
for name, key in zip(results, keys):
    csv_file = pd.read_csv(name+".csv")
    result = get_result_by_conditions(csv_file, result_key=key)
    result = add_mean(result)
    
    print(name)
    print()
    print_result(result, multiplier=1, round_num=1)
    
    try:
        os.mkdir("Average speed by Conditions")
    except:
        pass
    result = pd.DataFrame(result)
    result = result.round(1)
    result.to_csv(f"Average speed by Conditions/{name}.csv")
    print()

no data training_conditions out_of_track
no data training_conditions is_collision
no data training_conditions nospeed
sample_result

                     training_conditions         new_weather            new_city    new_city_weather
                 100                 5.3                 5.4                 3.9                 4.3
                 200                 5.8                 5.8                 4.1                 4.4
                 300                 5.7                 6.0                 4.1                 4.5
                 400                 5.8                 6.0                 4.0                 4.5
                 500                 5.9                 6.1                 4.0                 4.4
                 600                 5.9                 6.1                 4.3                 4.3
                  -1                 6.0                 6.2                 4.2                 4.5
                mean                 5.8                 6.

# Average speed by Offroads

In [19]:
keys = ["average_speed" for _ in range(len(results))]

In [20]:
for name, key in zip(results, keys):
    csv_file = pd.read_csv(name+".csv")
    result = get_result_by_offroads(csv_file, result_key=key)
    result = add_mean(result)
    
    print(name)
    print()
    print_result(result, multiplier=1, round_num=1)
    
    try:
        os.mkdir("Average speed by Offroads")
    except:
        pass
    result = pd.DataFrame(result)
    result = result.round(1)
    result.to_csv(f"Average speed by Offroads/{name}.csv")
    print()

sample_result

                               Offroad_1           Offroad_2           Offroad_3           Offroad_4           Offroad_5           Offroad_6           Offroad_7           Offroad_8
                 100                 5.4                 5.4                 5.4                 5.5                 3.5                 3.0                 3.5                 3.5
                 200                 5.8                 5.6                 5.8                 5.6                 3.3                 2.9                 3.4                 3.7
                 300                 6.0                 5.8                 5.7                 6.0                 3.7                 2.9                 3.5                 3.7
                 400                 6.0                 5.4                 5.7                 5.6                 3.6                 2.8                 3.9                 3.6
                 500                 6.1                 5.6                 5.6

# Infraction Test

In [21]:
keys = ["current_step"]

In [22]:
for name, key in zip(results, keys):
    csv_file = pd.read_csv(name+".csv")
    result = get_result_by_conditions(csv_file, result_key=key)
    result = add_mean(result)
    
    print(name)
    print()
    print_result(result, multiplier=1, round_num=1)
    print()

no data training_conditions out_of_track
no data training_conditions is_collision
no data training_conditions nospeed
sample_result

                     training_conditions         new_weather            new_city    new_city_weather
                 100               190.1               185.0               110.3               101.4
                 200               342.9               342.8               157.2               144.2
                 300               519.5               491.7               177.8               189.0
                 400               677.3               657.4               235.5               231.1
                 500               833.8               806.3               234.5               261.8
                 600               994.9               966.2               285.3               282.8
                  -1              1980.1              1906.6               629.5               732.7
                mean               791.2               765.